This notebook parse the catalogue into atomic artworks.

In [1]:
from bs4 import BeautifulSoup

from tqdm.autonotebook import tqdm
import pandas as pd
import numpy as np

from string import punctuation
from enum import Enum
import re

import functools

dashes = ["-","—","--"]

C:\SysNumNoSpace\Anaconda3\lib\site-packages\tqdm\autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


# Raw OCR to items parsing

In [2]:
def parse_page(words_list,target_list,trash_list,parity):
    """ 
    Parse a page into a list of entries. 
  
    Parameters: 
    words_list : list of OCR blocks that represent words.
    target_list : list that gather the valid entries.
    trash_list : control list that gather items that are rejected (header of a page, caption of illustration, etc).
    parity : parity of the page.
  
    """
    
    param = {"even":{"left_threshold":800,"right_threshold":1150},"odd":{"left_threshold":550,"right_threshold":900}}
    artwork_headers = ["*","S","S.","PEINTURES"]
    page_header = set(["MUSÉE","NATIONAL","DU","LOUVRE"]) #"Paris" not included because OCR may give "PARIS" or "PARIS,"
    separator_dashes = ["-","—"]
    
    
    previous_position = float("inf")
    prev_subcontent = False
    
    artwork = []
    
    for word in words_list:
        
        current_position = int(word["HPOS"])
        
        cond_position_left_first_word = current_position < param[parity]["left_threshold"]
        cond_position_right_first_word = previous_position > param[parity]["right_threshold"]
        cond_format_first_word = (word["CONTENT"].strip("()").isdigit() or word["CONTENT"] in artwork_headers)
        
        #Check if new artwork begins here
        if cond_position_left_first_word and cond_position_right_first_word and cond_format_first_word:          
            
            
            #Differentiate real artwork description from page header and picture legends
            cond_artwork_len = len(artwork) > 0
            cond_artwork_vs_header = not page_header <= set(artwork)
            cond_artwork_dash = any(dash in artwork for dash in separator_dashes) # Bad condition - not use
            cond_artwork_capital = not all(word.isupper() or len(word) == 1 \
                                           or all(char in punctuation or char.isdigit() for char in word)for word in artwork)
            
            if cond_artwork_len and cond_artwork_capital:
                target_list.append(artwork)
            else:
                trash_list.append(artwork)
                
            artwork = []
       
        if not prev_subcontent:
            if not word.has_attr("SUBCONTENT"):
                artwork.append(word["CONTENT"])
            else:
                artwork.append(word["SUBCONTENT"])
                prev_subcontent = True
        else:
            prev_subcontent = False
        
        previous_position = current_position
    

In [3]:
catalogue_index = range(21,193)

filename_alto = "data/alto_louvre_1923/alto_louvre_1923_{}.xml"

words_pages_even = []
words_pages_odd = []

for i in tqdm(catalogue_index):
    xml_file = open(filename_alto.format(i),"rb")
    xml = xml_file.read()
    xml_file.close()
    
    soup = BeautifulSoup(xml,"lxml-xml")
    
    l = []
    
    for word in soup.findAll("String"):
        l.append(word)
    
    if i%2 == 0:
        words_pages_even.append(l)
    else:
        words_pages_odd.append(l)

raw_artworks_pages = []
trash_list = [] # Detect artworks that would have been thrown away by error when OCR parsing.

for i in range(max(len(words_pages_even),len(words_pages_odd))):
    
    odd = []
    even = []
    trash_odd = []
    trash_even = []
    
    if i < len(words_pages_odd):
        parse_page(words_pages_odd[i],odd,trash_odd,"odd")
    
    raw_artworks_pages.append(odd)
    trash_list.append(trash_odd)
    
    if i < len(words_pages_even):
        parse_page(words_pages_even[i],even,trash_even,"even")

    raw_artworks_pages.append(even)
    trash_list.append(trash_even)

In [4]:
for n, page in enumerate(raw_artworks_pages):
    print("\nPAGE",n+1)
    for artwork in page:
        print(artwork)



PAGE 1
['*', 'Alaux,', 'Jean', "(1786-1864').", '—', 'Poussin', 'arrivant', 'de', 'Rome', 'est', 'présenté', 'par', 'Richelieu', 'au', 'roi', 'Louis', 'XIII.', 'Plafond.', 'Ire', 'S.', 'de', 'la', 'Céramique', 'antique']
['*', '-', 'Douze', 'médaillons', "d'or", 'représentant', 'les', 'travaux', "d'", 'Hercule.', 'Voussures.', '—']
['2', 'Aligny,', 'Claude-François-Théodore', 'Caruelle', "d'.", '(r', '798-', '1871).', '-', 'Une', 'villa', 'itaJienne.', 'VIII-E']
['S.', 'N°', 'Amaury-Duval.', '(1808-1885).', '—', 'Portrait', 'de', 'Mille', 'X..', 'Palier', 'esc.T.T.']
['9', 'Aved.', 'André-Joseph.', '(1702-1766).', '—', 'Portrait', 'du', 'marquis', 'de', 'Mirabeau.', 'XVI-S']
['*', 'Balze,', 'Paul-Jean-Etienne.', '(1815-1884)', 'et', 'Balze,', 'Ray-', 'mond-Joseph-Antoine.', '(', '1818-1909).', 'Homère', 'déifié,', "d'après", 'Ingres.', 'Plafond.', 'Musée', 'Charles', 'X', 'Salle', 'IX', 'Baptiste,', 'J.-B.', 'Monnoyer', 'dit.', 'Voir', 'Monnoyer.']
['2800', 'Barye,', 'Antoine-Louis.',

['130', '—', 'Portrait', "d'Elisabeth", "d'Autriche,", 'reine', 'de', 'France,', 'femme', 'de', 'Charles', 'IX', '(Attribué).', 'XI']
['131', '—', 'Portrait', 'de', 'François', 'de', 'Lorraine,', 'duc', 'de', 'Guise,', 'tué', 'au', 'siège', "d'Orléans", 'en', '1563.', '(réplique', 'présumée)', '.,.,.', 'XI']
['131', 'a', '—', 'Portrait', 'de', 'Henri', 'II,', 'en', 'buste', '(réplique', 'présumée', '1', 'XI']
['132', '—', 'Portrait', 'de', 'Charles', 'IX,', 'en', 'buste,', '(réplique', 'présumée).', 'XI']
['133', '—', 'Portrait', "d'Elisabeth", "d'Autriche,", 'reine', 'de', 'France,', 'femme', 'de', 'Charles', 'IX,', 'en', 'buste,', 'avec', 'toque', 'à', 'plumes', '(réplique', 'présumée)', '-', 'XI']
['133', 'a', '—', 'Portrait', 'de', 'Claude', 'de', 'Beaune,', 'dame', 'de', 'Château-', 'brun,', 'duchesse', 'de', 'Roannois', '(réplique', 'présumée)', 'XI']
['133', 'b', '—', 'Portrait', 'de', 'Jean', 'Babou,', 'seigneur', 'de', 'la', 'Bour-', 'daisière,', 'maréchal', 'général', 'de', "

['(37)', '—', 'Intérieur', 'de', 'cour', 'rustique.,.', '—', 'IV', '(38i', '—', 'Le', 'Christ', 'au', 'prétoire', '—', 'galerie', '158>', 'Degas,', 'Edgar.', '1', '1', 'g', '31', '-', '1', 'l)', 'l', '7).', '—', 'Course', 'de', 'Gentlemen', '-', 'Avant', 'le', 'départ', 'Coll.', 'Camondo', '(159^', '—', 'La', 'femme', 'à', 'la', 'potiche.', ',', '-']
['(160)', '—', 'Le', 'foyer', 'de', 'la', 'danse', 'à', "l'Opéra", 'de', 'la', 'rue', 'Le', 'Peletier', '.,', '-', '1I6I)', '—', 'Le', 'pédicure.', '-']
['162)', '—', 'Répétition', "d'un", 'ballet', 'sur', 'la', 'scène', '-']
['163)', '—', 'Classe', 'de', 'danse', '-']
['1164)', '—', "L'absinthe.", '—']
['165)', '—', 'Devant', 'les', 'tribunes.', ',', '-']
['1166', ')', '—', 'Aux', 'courses.', 'Jockeys', 'amateurs', 'près', "d'une", 'voiture', '—']
['167)', '—', 'Danseuses', 'montant', 'un', 'escalier', '—']
['168)', '—', 'Les', 'repasseuses.', '—']
['217)', '—', 'La', 'Danseuse', 'au', 'bouquet', 'saluant', 'sur', 'la', 'scène.', 'Pastel'

['291', 'a', '—', 'Le', 'vœu', 'à', "l'amour.,.", 'XVI-S']
['293', '—', 'Les', 'baigneuses..,.', '1-0']
['294', '—', 'Bacchante', 'endormie.,.', 'I-E']
['295', '—', 'La', 'chemise', 'enlevée..', '-', '1-0']
['296', '—', 'La', 'musique.', 'Portrait', 'de', 'M.', 'de', 'la', 'Bretèche', '1-0']
['297', '—', "L'étude.", '1-0']
['298', '—', "L'inspiration.", ',', ',', ',', ',', '1-0']
['299', '—', 'Figure', 'de', 'fantaisie.', 'Portrait', 'de', "l'abbé", 'de', 'St', 'Non', '1-0', '2Q"', 'FRAGONARD', '298', 'FRAGONARD']
['300', 'Fragonard,', 'Jean-Honoré.', '(1732-1806).', '—', 'Jeune', 'femme', '1-0']
['301', '—', "L'orage.", '1-0']
['303', '—', 'Sganarelle', 'dans', 'le', 'Médecin', 'malgré', 'lui', ':', 'Acte', '1,', 'scène', 'VI', '(Peut-être', 'par', 'Watteati)', 'I-E']
['S.', 'N"', '—', "L'enfant", 'aux', 'fleurs', 'Coll.', 'Schlichting']
['S.', 'X"', '—', 'Le', 'songe', "d'amour", 'du', 'guerrier.', ',', ',', '—']
['S.', 'X"', '—', 'Le', 'nid', "d'oiseaux.", '—']
['304', 'Fréminet,', 

['372', 'd', '—', "L'enfant", 'à', 'la', 'poupée.', 'XVI-S']
['372', 'e', '—', "L'effroi.", 'XVI-S']
['373', '—', 'Portrait', 'du', 'peintre', 'Etienne', 'Jeaurat.,.', 'XVI-E']
['373', 'a', '—', 'Portrait', "d'homme.", ':', 'XVI-E']
['373', 'b', '—', 'Portrait', 'du', 'médecin', 'Duval.', 'IVe', 'S.', 'Meubles', 'XVIIIe']

PAGE 34
['374', 'Greuze,', 'Jean-Baptiste.', '(1725-1805).', '—', 'Tête', 'de', 'jeune', 'fille;', 'étude.', 'XVI-N']
['375', '-', 'Tête', 'de', 'jeune', 'fille;', 'étude', 'XVI-N']
['376', '—', 'Tête', 'de', 'jeune', 'fine.', 'I-E']
['377', '—', 'Danaë', '1-0', '■', '379', '-', 'Portrait', 'de', 'Fabre', "_d'Eglantine", 'I-E']
['381', '-', 'Son', 'portrait,', 'par', 'lui-même.', 'XVI-E']
['382', '-', 'Son', 'portrait,', 'par', 'lui-même.', 'L-E']
['S.', 'N°', '—', 'Portrait', "d'Edouard", 'Bertin.>', 'XYPS']
['S.', 'N°', '-', "L'innocence", 'enchaînée', 'par', 'les', 'Amours,', 'et', 'suivie', 'du', 'Repentir.', ':', 'Coll.', 'Schlichting']
['S.', 'No', '-', 'Portra

...

['2202', 'b', '-', 'Triptyque:', 'La', 'Vierge', 'et', "l'Enfant", 'Jésus,', 'le', 'dona-', 'teur', 'Jan', 'de', 'Sedano', 'et', 'son', 'fils', 'sous', 'la', 'pro-', 'tection', 'de', 'saint', 'Jean-Baptiste,', 'la', 'femme', 'du', 'donateur', 'accompagnée', 'de', 'saint', "Jean-l'Evan-", 'géliste.', '—', 'Revers', ':', 'Adam', 'et', 'Eve', 'XXIX', '-']

PAGE 132
['1958', 'Diepenbeeck,', 'Abraham', 'van', '(1596-1675).', '—', 'Clélie', 'passant', 'le', 'Tibre', 'et', 'emmenant', 'avec', 'elle', 'ses', 'compagnes.', 'XXXIV']
['1960', 'Duchâtel,', 'François.', '(1625,', 'ou', 'peut-être', '1616-1694,', 'ou', 'peut-être', '1679).', 'Portrait', "d'un", 'cavalier', 'et', 'de', 'deux', 'autres', 'personnages.', 'XVII']
['1961', 'Dyck,', 'Anthonis', 'van', '(1599-1641).', '-', 'La', 'Vierge', 'et', "l'Enfant", 'avec', 'les', 'trois', 'repentants.', 'XVII']
['1962', '—', 'La', 'Vierge', 'aux', 'donateurs', 'XVII']
['1963', '—', 'Le', 'Christ', 'pleuré', 'par', 'la', 'Vierge', 'et', 'par', 'les'

['2109', 'Rubens,', 'Petrus-Paulus.', '(1577-1640).', '—', 'Portrait', 'de', 'Marie', 'de', 'Médicis,', 'sous', 'les', 'traits', 'de', 'la', 'France', '(Ecole', 'de)', 'XXXV']
['2110', '—', 'Le', 'triomphe', 'de', 'la', 'Vérité', 'et', 'les', 'Parques', 'filant', 'la', 'destinée', 'de', 'Marie', 'de', 'Médicis;', 'esquisse', 'des', 'deux', 'compositions', 'de', 'la', 'Galerie', 'du', 'Luxem-', 'bourg', 'VI', 'tr.', 'E-S']
['2111', '—', 'Portrait', 'du', 'baron', 'Henri', 'de', 'Vicq,', 'ambassadeur', 'des', 'Pays-Bas', 'à', 'la', 'Cour', 'de', 'France', 'VI', 'tr.', 'E-S']
['2112', '—', 'Portrait', "d'Anne", "d'Autriche,", 'reine', 'de', 'France.', '(Faussement', 'désigné', 'autrefois', 'sous', 'le', 'nom', "d'Elisabeth", 'de', 'France,', 'fille', 'de', 'Henri', 'IV)', 'VI', 'tr.', 'E-N']
['2113', '—', 'Portrait', "d'Hélène", 'Fourment,', 'seconde', 'femme', 'de', 'Rubens,', 'avec', 'son', 'fils', 'François', 'et', 'sa', 'fille', 'Claire-Jeanne.', ',', ',', ',', ',', ',', ',', ',', 'VI

['2337', 'Brekelenkam,', 'Quieringh', 'Gerritsz.', '(vers', '1620-1668).', '—', 'La', 'consultation', 'XX']
['2338', 'Ceulen,', 'Janson', 'ou', 'Janssens', 'van', '(', '1593-versi664).', '—', '-', 'Portrait', "d'homme", 'XXIII']
['2339', '—', 'Portrait', 'de', 'femme', 'I-N']
['S.', 'No', 'Claess', 'ou', 'Claessens,', 'Jacobus,', 'dit', 'Jacob', 'van', 'Utrecht.', '(travaillait', 'entre', '1S22', 'et', '1528).', '—', 'La', 'jeune', 'femme', 'à', "l'œillet..,.,.", 'Coll.', 'Arconati', 'Visconti']
['2341', 'Cuyp,', 'Aelbert.', '(1620-1691).', '—', 'Paysage;', 'pâtre', 'gardant', 'des', 'vaches', 'et', 'jouant', 'du', 'chalumeau.', 'VI', 'tr.', 'F-N']
['2342', '—', 'Le', 'départ', 'pour', 'la', 'promenade.,.', 'o.,.', 'VI', 'tr.', 'F-N']
['2343', '—', 'La', 'promenade', 'des', 'cavaliers.', 'VI', 'tr.', 'F-N']
['2344', '—', 'Portraits', "d'enfants.", 'Les', 'enfants', 'à', 'la', 'chèvre', 'XXVII']

PAGE 149
['2346', 'Dekker', 'ou', 'Decker,', 'Cornelis', 'Gerritsz.', '(', '?', '-1678)', '

['2508', 'Ostade,', 'Isack', 'van.', '(1621-1649).', '-', 'Halte', 'de', 'voya-', 'geurs', 'à', 'la', 'porte', "d'une", 'hôte]]erie.', 'XXII']
['2509', '—', 'La', 'halte,.', 'XXV']

PAGE 159
['PARIS,', 'MUSÉE', 'NATIONAL', 'DU', 'LOUVRE', '/j;-{}']
['2511', 'Ostade,', 'Isack', 'van.', '(1621-1649).', '—', 'Un', 'canal', 'gelé,', 'en', 'Hollande.', 'XXVI']
['2513', '—', 'Le', 'toit', 'à', 'porcs.', 'o.', 'XX']
['2515', '—', 'P.lys:lge:', 'effet', "d'hiver.", 'XX']
['2515', '.7', 'Palamedesz,', 'Anthonie.', 'vers', '1601-1073).', '—', 'Portrait', "d'homme", 'âgé', "de';!", 'ans.', 'XXIV', '253X', 'REMBRANDT', '2^40', 'REMBRANDT', '2527', 'PAULUS', 'POTTHR']
['2516', 'Poel,', 'Egbert', 'van', 'der.', '(1621-1664).', '—', 'La', 'maison', 'rustique', 'XXIII']
['2517', '—', 'Devant', 'une', 'chaumière.', ',', 'XX']
['2519', 'Poelenburgh,', 'Cornelis', 'van.', ',i5s6-i667).', '-', 'Le', 'pâ-', 'turage.', 'XXIII']
['2520', '-', 'Les', 'baigneuses', 'XXIII']
['2521', '—', 'Femmes', 'sortant', '

['2720', 'Holbein,', 'Hans,', 'le', 'jeune.', '(1497-', '1543).', '—', 'Portrait', "d'homme", 'âgé', 'de', '25', 'ans.', '(École', 'de)', 'XXXIII']
['S.', 'Nu', '—', 'Portrait', "d'homme,", '(vers', '1516)', 'XXXIII']
['S.', 'N°', '—', 'Portrait', 'de', 'la', 'femme', 'de', "l'artiste.", '\\Dessin).', 'XXXIII']
['S.', 'N°', '—', 'Le', 'triomphe', 'de', 'la', 'richesse.', '\\Dcssin)', 'XXXIII']
['S.', 'N°', '—', 'Deux', 'dessins:', '1°', 'Etudes', 'de', 'mains,', '2"', 'Etude', 'de', 'main', 'et', 'croquis', 'du', 'visage', "d'Erasmc", 'XXXIII']

PAGE 171
['S.', 'NI)', 'Maître', 'de', 'Gôttingen.', '(vers', '1420).', '—', 'Le', 'Christ', 'des-', 'cendu', 'de', 'la', 'croix.', '(Genre', 'du)', 'XXXII']
['S.', 'N°', 'Maître', 'du', 'Livre', 'de', 'Raison,', '(travaillait', 'vers', '[460', '-IC:;IO).', '-', 'Saint', 'Georges.', 'Nouv.', 'Acq.']
['2739', 'Maître', 'de', 'Messkirch.', '(XVlme', 'siècle).', '—', 'Le', 'Christ', 'devant', 'Pilate.', 'XXXII']
['2737', 'Maître', 'de', 'Saint', '

# Items to artwork informations parsing

In [5]:
class Artwork():
    
    def __init__(self):
        self.author = ""
        self.number = ""
        self.life = ""
        self.title = ""
        self.position = ""
        self.note = ""
        self.raw = ""
    
    def todict(self):
        return {"number": self.number,"author":self.author,"life":self.life,\
                "title":self.title,"position":self.position, "raw":self.raw}

In [6]:
def merge_position(raw_position,current_position):
    """ 
    Fill a raw position containing a dash based on the current known position
  
    Parameters: 
    raw_position : the position to clean
    current_position : the known position
  
    Returns: 
    string : cleaned position 
  
    """
    
    dashes = ["-","—"]
    regex_dashes = "^- |^— | - | — |^-$|^—$| -$| —$"
    
    if re.search(regex_dashes,raw_position) == None:
        return raw_position
    
    raw_position = raw_position.split(" ")
    current_position = current_position.split(" ")
    
    raw_position.reverse()
    current_position.reverse()
    
    new_position = []
    
    if len(current_position) >= len(raw_position):
        i = 0
        while i < len(current_position):
            if i < len(raw_position) and raw_position[i] not in dashes:
                new_position.append(raw_position[i])
            else:
                new_position.append(current_position[i])
            i += 1
    else:
        new_position = raw_position
    
    new_position.reverse()
    
    return " ".join(new_position)

In [7]:
artwork_processed_list = []

for page in tqdm(raw_artworks_pages):
    
    current_author = ""
    current_life = ""
    current_position = ""
    
    for raw in page:
        
        art = Artwork()
        
        raw_text = " ".join(raw)
        
        art.raw = raw_text
        
        
        #Searching for number
        regex_num = "^\* |^\(?\d+\)? ?[a-z]? |^S\.? [NXo].? "

        for i, match in enumerate(re.finditer(regex_num,raw_text)):
            
            if i == 0 :
                (start,end) = match.span()
                art.number = raw_text[start:end]
                raw_text = raw_text[end:]
                
            else:
                raise ValueError("Multiple matches found for beginning of string")
        
        
        #Searching for author and his life
        regex_dash = "^- |^— "
        is_new_author = True
        
        for i, match in enumerate(re.finditer(regex_dash, raw_text)):
            if i == 0:
                is_new_author = False
                (start,end) = match.span()
                art.author = current_author
                art.life = current_life
                raw_text = raw_text[end:]
            else:
                raise ValueError("Multiple matches found for beginning of string")
        
        #First case standard, second case when only a century is indicated, third and fourth for OCR degenerated cases
        regex_life = "[ .]\([^()]+\d+[^()]+\)\.? |[ .]\([^()]+[IVX]+.?.? [^()]+\)\.? | .?.\d+-\d+..? | .\S*\d\S*-\S*\d\S*. "
        if is_new_author:
            #Case of anonymous authors
            regex_anonymous = "^Ecole .+ — |^École .+ — |^Première moitié du .+ — |^[XVI]+e Siècle.+ — |^Fin du .+ — "
            anonymous_match = re.search(regex_anonymous,raw_text)
            if anonymous_match != None:
                art.author = "<anonyme>"
                (start,end) = match.span()
                raw_text = raw_text[end:]
            else:    
                #Case of known authors
                for i, match in enumerate(re.finditer(regex_life, raw_text)):
                    if i == 0:
                        (start,end) = match.span()
                        art.life = raw_text[start:end]
                        art.author = raw_text[:start]
                        raw_text = raw_text[end:]
                    
                    
        #Searching for position
        #Three possible matches : dash, header (specials rooms), roman number (numbered rooms)
        
        #Eliminate some OCR parasits
        raw_text = re.sub("^\.|^\,|[.,][.,]+","",raw_text)
        raw_text = re.sub("--","-",raw_text)
        raw_text = raw_text.strip(" ")
        
        #Eliminate the dash at the begining of the line if it is still here
        for i, match in enumerate (re.finditer(regex_dash, raw_text)):
            if i == 0:
                (start, end) = match.span()
                raw_text = raw_text[end:]
            else:
                raise ValueError("Multiple matches found for beginning of string")
        
        
        regex_roman = "[1IXV]+-?.?$|[ABCD]-.$|[1IVX]+ tr\.? "
        roman_number_matches = re.finditer(regex_roman,raw_text)
        
        regex_header = " Ire | Palier | Musée | Coll\.? | Marine| Ga[li]\.? | Vest\.? | Esc\.? | Céramique | S\.? |" +\
    "T\.? T\.?$|E\.? F\.? | non exposé$| Nouv\.? Acq\.?$| Vestibule | Galerie | Ie? S\.? | IIe? S\.? | IIIe? S\.? | IVe? S\.? "
        header_matches = re.finditer(regex_header,raw_text)
        
        regex_dashes = " - | — | -$| —$"
        dash_matches = re.finditer(regex_dashes,raw_text)
        
        #Find separation. header has priority over dash, which has priority over roman number
        found_sep = False
        found_dash = False
        separator = len(raw_text)-1
        
        for i, match in enumerate(header_matches):
            if i == 0:
                found_sep = True
                separator = match.span()[0]
        
        if not found_sep:
            for i, match in enumerate(dash_matches):
                found_sep = True
                separator = match.span()[0]               
        
        if not found_sep:
            for i, match in enumerate(roman_number_matches):
                if i == 0:
                    found_sep = True
                    separator = match.span()[0]
        
        art.title = raw_text[:separator]
        
        if found_sep:
            art.position = merge_position(raw_text[separator:].strip(" "),current_position)
        
            
        current_author = art.author
        current_life = art.life
        current_position = art.position
        
        artwork_processed_list.append(art.todict())

In [8]:
artworks_df = pd.DataFrame(artwork_processed_list)

In [9]:
#Feedback loop
#for row in artworks_df[artworks_df.life.apply(lambda x : re.search("\d",x) == None)].iterrows():
#for row in artworks_df.iterrows():
    #print(row)

artworks_df.sample(20)

,number,author,life,title,position,raw
257,158,"Coypel, Noël.",(1628-1707).,Ptolémée Philadelphe donnant la liberté aux ju...,XIV-N,158 — Ptolémée Philadelphe donnant la liberté ...
629,*,"Guérin, Pierre-Narcisse, baron.",(1774-1833).,Deux génies de l'immortalité; médaillon du pla...,S. des Antonins,* — Deux génies de l'immortalité; médaillon du...
40,S. N°,"Borowikoffsky, Wladimir.",(1757-1825).,Portrait de l'impératrice Elisabeth de Russie ...,Coll. Schlichting,"S. N° Borowikoffsky, Wladimir. (1757-1825). — ..."
716,434,"Jouvenet, Jean.",(1614-1717).,La résurrection de Lazare,XIV-S,434 — La résurrection de Lazare XIV-S
1636,1308,"PARIS, MUSÉE NATIONAL DU LOUVRE U5 Ghirlandajo...",(1632-170=;).,Le mariage de la Vierge.,1-0,1308 — Le mariage de la Vierge. 1-0
381,211,"Delacroix, Ferdinand-Victor-Eugène.",(1798-1863).,Noce juive au Maroc,VIII-N,211 - Noce juive au Maroc VIII-N
2128,1991,"Franck ou Francken, Frans, dit le jeune.",i 1581-1642 —,"La Passion. , , , o.",XXXIV,"1991 — La Passion. , , , o. XXXIV"
48,37,"Boucher, François.",(1703-1770).,Vertumne et Pomone.,XVI-S,37 - Vertumne et Pomone. XVI-S
2080,1943,"Champaigne, Philippe de",(1602-1674).,Portrait de femme inconnue,XII,1943 — Portrait de femme inconnue XII
664,2947,"Huet, Paul.",(1803-1869).—L'inondation,Grande marée d'équinoxe aux environs de Hon- f...,E. F. XIXe S. 2e étage 1,2947 — Grande marée d'équinoxe aux environs de...


In [10]:
#Sample of some uncorrectely parsed paintings
artworks_df[artworks_df.author.apply(lambda p : len(p) == 0)].sample(10)

,number,author,life,title,position,raw
434,2860,,,Les deux rivales,T. T.,2860 — Les deux rivales T. T.
1141,S. N°,,,Portrait de Mme A. Fouques Duparc.,Marine 1,S. N° — Portrait de Mme A. Fouques Duparc. Mar...
1458,"S. N""",,,"Portrait présumé de Madame Louise, fille de Lo...",S. des Pastels,"S. N"" — Portrait présumé de Madame Louise, fil..."
651,410,,,CI,- La leçon de musique. XVI-S,410 CI - La leçon de musique. XVI-S
1339,950,,,"Vue de la ville et du port de Bayonne, prise d...",,"950 — Vue de la ville et du port de Bayonne, p..."
370,(224),,,Femme dans une baignoire s'épongeant la jambe ...,Coll. Camondo (159^ i36) La femme à la potiche...,(224) — Femme dans une baignoire s'épongeant l...
2153,2208,,,et 2209,— Adam et Eve (Revers de deux volets de trip- ...,2208 et 2209 — Adam et Eve (Revers de deux vol...
2326,2018 b,,,Religieux offrant son cœur à l'Enfant Jésus (G...,XXXI,2018 b — Religieux offrant son cœur à l'Enfant...
647,*,,,Plafond : Le Vésuve reçoit de Jupiter le feu q...,Céramique antique S. VIII,* - Plafond : Le Vésuve reçoit de Jupiter le f...
151,S. N°,,,"Le souffleur. , ,",XVI-S,"S. N° -- Le souffleur. , , XVI-S"


In [11]:
#Liste de problèmes:
# Gai. au lieu de Gal.
# Deux auteurs au lieu d'un
# ; au lieu de -
# Chiffre romain pas en fin de chaîne (Bourguignon)
# Erreur d'OCR qui se répercute (principalement des dates très dégénérées)
## Solution possible : parenthèse de fin et tiret souvent bien conservés, comme dans "nooo-tyyo)"
# Lettre post-numéro OCRisée en chiffre (87 1 au lieu de 87 b)
# Quelques mauvais split d'oeuvres (121 c)
# Parenthèse transformée en chevron (124>
# Oeuvres multiples séparées par des tirets
# Vestibule ou Galerie écrit en toutes lettres et pas abrégés.
# Manque une parenthèse gauche sur les numéros: 162)



for row in artworks_df\
[(artworks_df.author.apply(lambda p : len(p) == 0)) | (artworks_df.position.apply(lambda p : len(p) == 0))].iterrows():
#for row in artworks_df.iterrows():
    print("Raw@",row[1].raw)
    print("Numéro@",row[1].number)
    print("Auteur@",row[1].author)
    print("Vie@",row[1].life)
    print("Titre@",row[1].title)
    print("Position@",row[1].position)
    print("\n")

Raw@ S. N° — L'Odalisque, Victoire O'Murphy. ;
Numéro@ S. N° 
Auteur@ Boucher, François.
Vie@  (1703-1770). 
Titre@ L'Odalisque, Victoire O'Murphy. 
Position@ 


Raw@ S. N° — Le sommeil des amours —
Numéro@ S. N° 
Auteur@ Boucher, François.
Vie@  (1703-1770). 
Titre@ Le sommeil des amours
Position@ 


Raw@ S. N° — Mme de Pompadour. Pastel. (Attribué). -
Numéro@ S. N° 
Auteur@ Boucher, François.
Vie@  (1703-1770). 
Titre@ Mme de Pompadour. Pastel. (Attribué).
Position@ 


Raw@ 45 FR. BOUCHER 5° FR. BOUCHER
Numéro@ 45 
Auteur@ 
Vie@ 
Titre@ FR. BOUCHER 5° FR. BOUCHE
Position@ 


Raw@ 2932 Cals, Adolphe-Félix. i i S i o- i SSo). — Femmes effilant de l'étoupe E. F. XIXe S.2e étage II
Numéro@ 2932 
Auteur@ 
Vie@ 
Titre@ Cals, Adolphe-Félix. i i
Position@ S i o- i SSo). — Femmes effilant de l'étoupe E. F. XIXe S.2e étage II


Raw@ 2933 — Lard et harengs —
Numéro@ 2933 
Auteur@ 
Vie@ 
Titre@ Lard et harengs
Position@ S i o- i SSo). — Femmes effilant de l'étoupe E. F. XIXe S.2e étage II


Raw@

Position@ - Saint Bruno refuse l'archevêché de Reggio que lui offre Urbain II. XIV-S


Raw@ 581 — Saint Bruno en prière dans sa cellule rustique, non exposé
Numéro@ 581 
Auteur@ 
Vie@ 
Titre@ Saint Bruno en prière dans sa cellule rustique,
Position@ non exposé


Raw@ 582 — Rencontre de saint Bruno par le comte Roger —
Numéro@ 582 
Auteur@ 
Vie@ 
Titre@ Rencontre de saint Bruno par le comte Roger
Position@ non exposé


Raw@ 583 — Apparition de saint Bruno au comte Roger —
Numéro@ 583 
Auteur@ 
Vie@ 
Titre@ Apparition de saint Bruno au comte Roger
Position@ non exposé


Raw@ 584 — Mort de saint Bruno. XIV-N
Numéro@ 584 
Auteur@ 
Vie@ 
Titre@ Mort de saint Bruno. 
Position@ XIV-N


Raw@ 585 — Saint Bruno est enlevé au ciel. non exposé
Numéro@ 585 
Auteur@ 
Vie@ 
Titre@ Saint Bruno est enlevé au ciel.
Position@ non exposé


Raw@ 586 — Saint Bruno et ses compagnons distribuent tous leurs biens aux pauvres. Esquisse.,. XIII
Numéro@ 586 
Auteur@ 
Vie@ 
Titre@ Saint Bruno et ses compagnons dis

Raw@ S. N" — Guerre de Troie. La prise de la ville. Dessin aquarellé.,. —
Numéro@ S. N" 
Auteur@ 
Vie@ 
Titre@ Guerre de Troie. La prise de la ville. Dessin aquarellé
Position@ 


Raw@ 1010 -- Portrait de Jean d'Albon, seigneur de Saint- André, âgé, en manteau brun à revers de fourrure blanche. , , , : , XI
Numéro@ 1010 
Auteur@ 
Vie@ 
Titre@ Portrait de Jean d'Albon, seigneur de Saint- André, âgé, en manteau brun à revers de fourrure blanche. , , , : , 
Position@ XI


Raw@ 1011 - Portrait de Jean de Bourbon-Vendôme, comte d'Enghien, jeune, avec barbe blonde naissante. XI
Numéro@ 1011 
Auteur@ 
Vie@ 
Titre@ Portrait de Jean de Bourbon-Vendôme, comte d'Enghien, jeune, avec barbe blonde naissante. 
Position@ XI


Raw@ 1011 a — Portrait présumé de Louise de Rieux, Marquise d'Elbeuf. XI
Numéro@ 1011 a 
Auteur@ 
Vie@ 
Titre@ Portrait présumé de Louise de Rieux, Marquise d'Elbeuf. 
Position@ XI


Raw@ 1011 b — Portrait supposé de Henri de Bourbon, duc de Montpensier, en blanc. XI
Numéro@ 101

Position@ 


Raw@ 1728 — Un philosophe tourné à droite, une gourde à la ceinture. (Attribué) —
Numéro@ 1728 
Auteur@ Ribera, José de, dit L'Espagnolet.
Vie@  (158N-1652). 
Titre@ Un philosophe tourné à droite, une gourde à la ceinture. (Attribué)
Position@ 


Raw@ 1729 — Un philosophe tourné à droite, tenant un livre et un rouleau de papier. (Attribué). —
Numéro@ 1729 
Auteur@ Ribera, José de, dit L'Espagnolet.
Vie@  (158N-1652). 
Titre@ Un philosophe tourné à droite, tenant un livre et un rouleau de papier. (Attribué).
Position@ 


Raw@ S. Nu Theotocopuli, Domenico, dit El Greco. 15)8-1 (>14). - Portrait du roi Ferdinand. —
Numéro@ S. Nu 
Auteur@ Theotocopuli, Domenico, dit El Greco. 15)8-1
Vie@  (>14). 
Titre@ Portrait du roi Ferdinand.
Position@ 


Raw@ 128 PARIS, MUSÉE NATIONAL DU LOUVRE ÉCOLE FLAMANDE 2733 Alsloot, Denis van; voir Clerck, Hendrik de.
Numéro@ 128 
Auteur@ 
Vie@ 
Titre@ PARIS, MUSÉE NATIONAL DU LOUVRE ÉCOLE FLAMANDE 2733 Alsloot, Denis van; voir Clerck, Hendrik de
P

Raw@ S. N° — Portrait de Henri IV. (Ecole de). lIme salle de la Colonnade
Numéro@ S. N° 
Auteur@ Pourbus, Frans, dit le jeune.
Vie@  (1=169 ou 15 70-1622). 
Titre@ Portrait de Henri IV. (Ecole de). lIme salle de la Colonnad
Position@ 


Raw@ 2151 -cr Les oiseaux sur la branche. I-E
Numéro@ 2151 
Auteur@ 
Vie@ 
Titre@ -cr Les oiseaux sur la branche. 
Position@ I-E


Raw@ 2152 — Les singes voleurs de fruits,.,.,..,..,.,.. XXXV
Numéro@ 2152 
Auteur@ 
Vie@ 
Titre@ Les singes voleurs de fruits 
Position@ XXXV


Raw@ 2153 - Dispute de deux singes et d'un perroquet autour d'une corbeille de fruits. XXXV
Numéro@ 2153 
Auteur@ 
Vie@ 
Titre@ Dispute de deux singes et d'un perroquet autour d'une corbeille de fruits. 
Position@ XXXV


Raw@ 2156 -* L'enfant prodigue à table avec des courtisanes. VI tr. E-N
Numéro@ 2156 
Auteur@ 
Vie@ 
Titre@ -* L'enfant prodigue à table avec des courtisanes. 
Position@ VI tr. E-N


Raw@ 2157 - Les sept œuvres de miséricorde XVII
Numéro@ 2157 
Auteur@ 
Vie@ 
Titre@ 

In [12]:
#Rows with an empty field are always incorrectly parsed. Here is a lower bound for the percentage of failures.
print(len(artworks_df[artworks_df.author.apply(lambda p : len(p) == 0)])/len(artworks_df))

0.10668143226282761


In [13]:
#Join together words that were splitted in the catalogue due to line return.
artworks_df["title"] = artworks_df.title.apply(lambda text : re.sub("(\S)- (\S)","\\1\\2",text))

print(len(artworks_df))
print(len(artworks_df.replace("",np.nan).dropna()))

2709
2254


In [25]:
artworks_df.replace("",np.nan).dropna().sample(10)

,number,author,life,title,position,raw
560,340,"Géricault, Jean-Louis-André-Théodore.",(17° J - J 824).,Officiers de chasseurs à cheval de la garde im...,E. F. XIXe S. 1er étage II,340 — Officiers de chasseurs à cheval de la ga...
2312,2194,"Vos, Paul de.",(vers 1590-1678).,La mort du chevreuil,1-0,2194 — La mort du chevreuil 1-0
87,75,"Bourdon,Sébastien",.(1616-1671).,UnehaltedeBohémiens,XIV-N,"75 Bourdon,Sébastien.(1616-1671). UnehaltedeBo..."
2578,2559,"Ruisdael ou Ruysdael, Jacob van.",(1628 ou 1620- 1682).,"Paysage, dit le Buisson",VI tr. F-S,"2559 — Paysage, dit le Buisson VI tr. F-S"
1199,828,"Rousseau, Théodore.",1812-1867).,Lisière d'une forêt; esquisse,E. F. XIXe S. 1er étage 1,828 — Lisière d'une forêt; esquisse E. F. XIXe...
1681,1361,"Luini, Bernardino",(entre 1475 et 1480-1531 ou 1532).,Le Christ. Fresque de l'Oratoire de Greco Mila...,V,1361 — Le Christ. Fresque de l'Oratoire de Gre...
1165,794,"Rigaud y Ros, Hyacinthe.",(1059-1743).,Portrait d'homme âgé.,I-E,"794 Rigaud y Ros, Hyacinthe. (1059-1743). — Po..."
178,132,"Clouet, François, dit Jehannet, ou Janet.",(mort en 1572).,"Portrait de Charles IX, en buste, (réplique pr...",XI,"132 — Portrait de Charles IX, en buste, (répli..."
702,2954,"Isabey, Eugène.",(1804-1886).,Le pont,E. F. XIXe S. 2e étage 1,2954 — Le pont E. F. XIXe S. 2e étage 1
420,249,"Desportes, François.",(1661-1743).,"Son portrait, par lui-même, en chasseur.",XVI-N,"249 — Son portrait, par lui-même, en chasseur...."


In [15]:
clean_artworks_df = artworks_df.replace("",np.nan).dropna().drop("raw",axis = 1)
clean_artworks_df.to_csv("data/catalogue_artworks.csv")